# IMPORT

In [ ]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk

from tkinter import ttk
from tkinter import *

# Load Datasets

In [ ]:
data_path = 'datasets/Datafull train.csv'

df = pd.read_csv(data_path, delimiter=";")

df.head()

In [ ]:
# cek dimensi data sebelum masuk data preprocessing
df.shape

In [ ]:
df.nunique() # mengecek data unique pada setiap kolom

In [ ]:
df.info()

In [ ]:
df.dtypes

# Data PreProcessing

In [ ]:
df.isna().sum() # mengecek data yang kosong
df.duplicated().sum # mengecek data yang duplikat



In [ ]:
df = df.drop_duplicates() #menghapus data duplikat
df = df.fillna(df.mode().iloc[0])

In [ ]:
df.shape # cek dimensi data setelah dihapus data duplikat

# Exploratory Data Analys

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(data=df, x="char")
plt.title("Distribusi Label / Char")
plt.ylabel("Jumlah Koordinat")
plt.show()

In [ ]:
def plot_sample(row): 

    x = [row[f"{i}x"] for i in range(21)]
    y = [row[f"{i}y"] for i in range(21)]

    plt.Figure(figsize=(10, 8))
    plt.plot(x, y, marker="o")
    plt.gca().invert_yaxis()
    plt.title(f"Class: {row['char']}")
    plt.show()

In [ ]:
for label in df['char'].unique():
    sample = df[df['char'] == label].iloc[0]
    plot_sample(sample)

In [ ]:
x_cols = [c for c in df.columns if c.endswith('x')]
y_cols = [c for c in df.columns if c.endswith('y')]

plt.figure(figsize=(12, 4))
sns.boxplot(data=df[x_cols])
plt.title("Distribusi Koordinat X")
plt.show()

plt.figure(figsize=(12, 4))
sns.boxplot(data=df[y_cols])
plt.title("Distribusi Koordinat Y")
plt.show()

# Setup Untuk Pengambilan Citra / Gambar

In [ ]:
import cv2
import os
from datetime import datetime
import numpy as np

cap = cv2.VideoCapture(0)

ret, frame = cap.read()
root_folder = 'samples'
os.makedirs(root_folder, exist_ok=True)

NUM_POINTS = 20

print("Tekan 'C' untuk capture, 'q' untuk keluar")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membuka kamera")
        break
    
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_skin = np.array([0, 15, 60])
    upper_skin = np.array([30, 255, 255])
    
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    mask = cv2.GaussianBlur(mask, (5,5), 0)
    
    #fungsi mencari kontur
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    
    if contours:
        cnt = max(contours, key=cv2.contourArea)
        
        if cv2.contourArea(cnt) > 2000:
            idx = np.linspace(0, len(cnt) - 1, NUM_POINTS, dtype=int)
            points = cnt[idx].reshape(-1, 2)
            
            
            for (x, y) in points:
                cv2.circle(frame, (x,y), 4, (0, 0, 255), -1)
            
            print(points[:3])
    
    
    cv2.imshow("Camera GUI", frame)
    
    key = cv2.waitKey(1) & 0xff
    
    if key == ord('c'):
        filename = datetime.now().strftime("%Y%m%d_%H%M%S") + ".jpg"
        file_path = os.path.join(root_folder, filename)
        cv2.imwrite(file_path, frame)
        print("Gambar disimpan : ", file_path)
        
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# GUI PENGAMBILAN GAMBAR

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import * 

class GUI:
    def __init__(self, root, tabCtrl):
        self.root = root
        self.tabCtrl = tabCtrl
        self.root.geometry("800x650")
        self.root.title("GUI KNN")
        
        self.uilayout()
        
    def uilayout(self):
        
        # halaman pertama 
        window1 = ttk.Frame(self.tabCtrl, borderwidth=0)
        self.tabCtrl.add(window1, text="Input data KNN")
        tk.Label(
            window1,
            text="Input Data Gambar Untuk Menguji Algoritma K-Nearest-Neighbor",
            font=("Inter", 10, "bold"),
            bg="white",
        ).pack(anchor="w", pady=(10, 10), padx=(10, 10))
        
        
        
        #halaman kedua
        window2 = ttk.Frame(self.tabCtrl)
        self.tabCtrl.add(window2, text="Informasi Model")
        tk.Label(
            window2, 
            text="Informasi Model",
            font=("Inter", 10, "bold"),
            bg="white",
        ).pack(anchor="w", pady=(10, 10), padx=(10, 10))
        
        
        
        #halaman ketiga 
        window3 = tk.Frame(self.tabCtrl)
        self.tabCtrl.add(window3, text="Informasi Visualisasi Dataset")
        tk.Label(
            window3,
            text="Informasi Visualisasi Dataset",
            font=("Inter", 10, "bold"),
            bg="#f1f5f9"
        ).pack(anchor="w", padx=(10, 10), pady=(10, 10))
        
        
        
        
        
if __name__ == "__main__":
    root = tk.Tk()
    tabCtrl = ttk.Notebook(root)
    tabCtrl.pack(fill='both', expand=True)
    
    app = GUI(root, tabCtrl)
    root.mainloop()

In [ ]:
df = df[df['char'].str.match(r'^[A-Za-z]$')]
df['char'] = df['char'].str.upper()

In [ ]:
print("\n=== Encoding Kolom 'char' ===")
unique_chars = sorted(df['char'].unique())
char_to_int = {char: idx for idx, char in enumerate(unique_chars)}
df['char_encoded'] = df['char'].map(char_to_int)
print("Mapping label:", char_to_int)

In [ ]:
df.head()

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.x_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)
    
    def _predict(self, x):
        distances = np.sqrt(np.sum((self.x_train - x) ** 2, axis=1))
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = self.y_train[k_indices]
        unique, counts = np.unique(k_nearest_labels, return_counts=True)
        most_common = unique[np.argmax(counts)]
        return most_common
    
    def accuracy(self, y_true, y_pred):
        return np.sum(y_true == y_pred) / len(y_true)

In [ ]:
def split_train_test(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    n_samples = len(X)
    n_test = int(n_samples * test_size)
    
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Persiapan data
X = df.drop(['char', 'char_encoded'], axis=1).values
y = df['char_encoded'].values

A = np.array([319.000,45.000,285.000,76.000,378.000,95.000,233.000,121.000,7.000,155.000,238.000,175.000,186.000,191.000,4.000,205.000,80.000,217.000,196.000,228.000,76.000,239.000,121.000,249.000,223.000,259.000,146.000,270.000,73.000,282.000,231.000,294.000,312.000,305.000,357.000,315.000,360.000,325.000,241.000,336.000,329.000,358.000]
)
A = A.reshape(1, -1)  # Reshape A to match the input shape expected by the KNN model

# print(X[0])

# print(A)

# print("Dimensi X:", X.shape)
# print("Dimensi y:", y.shape)

X_train, X_test, y_train, y_test = split_train_test(X, y, test_size=0.2, random_state=42)

In [ ]:
classify = KNN(k=3)
classify.fit(X_train, y_train) 

# predictions = classify.predict(A)

predictions= classify.predict(A)

arr_alfa = df['char_encoded'].unique()
alfa = df['char'].unique()


checking_data = zip(arr_alfa, alfa)
for x,y in checking_data:
    if x == predictions:
        print('benar', y)
    else:
        print('salah', y)